In [1]:
import json
import math

In [2]:
f = open("compressed_tree.json")
tree = json.loads(f.read())
layers = tree["layers"]
classes = tree["classes"]
f.close()

In [3]:
field_width = {
	"src": 16,
	"dst": 16,
	"protocl": 8,
}

# Worst Case RMT

In [4]:
def worst_case_rmt(tree):
	rmt = []
	step = 0

	tcam_bits = 0
	ram_bits = 0

	for layer in layers:
		num_ranges = len(layers[layer])
		# assume that each range requires all of 2*k prefixes when performing prefix expansion
		# therefore there are 2*k * R for R ranges and width k
		num_prefixes = 2 * field_width[layer] * num_ranges
		prefix_width = field_width[layer]

		tcam = {
			"id": f"{layer}_range",
			"step": step,
			"match": "ternary",
			"entries": num_prefixes,
			"key_size": prefix_width
		}
		tcam_bits += num_prefixes * prefix_width

		# assume basic pointer reuse for metadata storage
		ram = {
			"id": f"{layer}_meta",
			"step": step,
			"match": "exact",
			"method": "index",
			"key_size": math.ceil(math.log2(num_ranges)),
			"data_size": len(classes)
		}
		ram_bits += num_ranges * len(classes)

		rmt.append(tcam)
		rmt.append(ram)

		step += 1

	return rmt, tcam_bits, ram_bits

x, tcam_bits, ram_bits = worst_case_rmt(tree)
f = open("worst_case_rmt.json", "w+")
f.write(json.dumps(x, indent=4))
f.close()

In [5]:
! command python3 ideal-rmt-simulator/sim.py naive_rmt.json
print(f"TCAM bits: {tcam_bits}")
print(f"RAM bits:  {ram_bits}")

TCAM mapping: 
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SRAM mapping: 
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
id mapping: 
[['dst_range', 'dst_meta'], ['src_range', 'src_meta'], ['protocl_range', 'protocl_meta'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
TCAM bits: 13184
RAM bits:  504


# Naive Range Expansion 

In [6]:
# shamelessly stolen from: https://github.com/autolyticus/range-to-prefix/blob/master/rangetoprefix.C

def int_to_bin(i, width):
	return bin(i)[2:].zfill(width)

def increment_dc(pfx):
	idx = pfx.find("*")
	if idx == -1:
		idx = len(pfx)
	idx = idx - 1
	#print(pfx, pfx[:idx])
	return pfx[:idx] + "*" + pfx[idx+1:]
	
def can_merge(pfx_a, pfx_b):
	pfx_a = pfx_a.replace("*", "")
	pfx_b = pfx_b.replace("*", "")
	return pfx_a[:-1] == pfx_b[:-1] and pfx_a[-1] != pfx_b[-1]

def merge(pfx_a, prefixes):
	pfx_a = increment_dc(pfx_a)
	prefixes[-1] = pfx_a

	for i in range(len(prefixes) - 2, -1, -1):
		if can_merge(prefixes[i], prefixes[i+1]):
			prefixes.pop()
			pfx = increment_dc(prefixes[i])
			prefixes[i] = pfx

def convert_range(lower, upper, width):
	prefixes = []
	prefix = int_to_bin(lower, width)
	prefixes.append(prefix)
	norm_upper = min(upper, 2**width-1)
	for i in range(lower+1, norm_upper+1):
		prefix = int_to_bin(i, width)
		if can_merge(prefix, prefixes[-1]):
			merge(prefix, prefixes)
		else:
			prefixes.append(prefix)
	return prefixes

In [7]:
def naive_rmt(tree):
	rmt = []
	step = 0

	tcam_bits = 0
	ram_bits = 0

	for layer in layers:
		num_prefixes = 0
		prefix_width = field_width[layer]
		# for each range in the layer, convert the ranges to prefixes using naive range expansion
		for r in layers[layer]:
			if r["min"] == None:
				r["min"] = 0
			elif r["max"] == None:
				r["max"] = 2 ** prefix_width
			prefixes = convert_range(r["min"], r["max"], prefix_width)
			r["prefixes"] = prefixes
			num_prefixes += len(prefixes)
			tcam_bits += len(prefixes) * prefix_width

		tcam = {
			"id": f"{layer}_range",
			"step": step,
			"match": "ternary",
			"entries": num_prefixes,
			"key_size": prefix_width,
			"ranges": layers[layer]
		}

		num_ranges = len(layers[layer])
		# assume no pointer reuse for metadata storage
		ram = {
			"id": f"{layer}_meta",
			"step": step,
			"match": "exact",
			"method": "index",
			"key_size": math.ceil(math.log2(num_ranges)),
			"data_size": len(classes)
		}
		ram_bits += num_ranges * len(classes)

		rmt.append(tcam)
		rmt.append(ram)

		step += 1

	return rmt, tcam_bits, ram_bits

x, tcam_bits, ram_bits = naive_rmt(tree)
f = open("naive_rmt.json", "w+")
f.write(json.dumps(x, indent=4))
f.close()


In [8]:
! command python3 ideal-rmt-simulator/sim.py naive_rmt.json
print(f"TCAM bits: {tcam_bits}")
print(f"RAM bits:  {ram_bits}")

TCAM mapping: 
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SRAM mapping: 
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
id mapping: 
[['dst_range', 'dst_meta'], ['src_range', 'src_meta'], ['protocl_range', 'protocl_meta'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
TCAM bits: 3320
RAM bits:  504


In [9]:
# for this technique, we note that given disjoint ranges [0,a][a,b],[b,c] ...
# then if using a TCAM that selects the first matching prefix, then [0,a],[0,b],[0,c] would be equivalent
# this is because if for some k<a, even though the range [0,b] could be selected, as long as the prefixes for [0,a] are before [0,b] then the correct prefix will still be selected

def priority_aware(tree):
	rmt = []
	step = 0

	tcam_bits = 0
	ram_bits = 0

	for layer in layers:
		num_prefixes = 0
		prefix_width = field_width[layer]
		# for each range, run the regular prefix expansion, and also the prefix expansion setting the minimum to 0
		# then check which set of prefixes would be better
		# we will assume the ranges are already disjoin and in the correct order
		for r in layers[layer]:
			if r["min"] == None:
				r["min"] = 0
			elif r["max"] == None:
				r["max"] = 2 ** prefix_width
			regular_prefixes = convert_range(r["min"], r["max"], prefix_width)
			zero_start_prefixes = convert_range(0, r["max"], prefix_width)

			if len(regular_prefixes) <= len(zero_start_prefixes):
				pfx_type = "exact"
				prefixes = regular_prefixes
			else:
				pfx_type = "zero"
				prefixes = zero_start_prefixes

			r["prefixes"] = prefixes
			r["prefix_type"] = pfx_type
			num_prefixes += len(prefixes)
			tcam_bits += len(prefixes) * prefix_width

		tcam = {
			"id": f"{layer}_range",
			"step": step,
			"match": "ternary",
			"entries": num_prefixes,
			"key_size": prefix_width,
			"ranges": layers[layer]
		}

		num_ranges = len(layers[layer])
		# assume no pointer reuse for metadata storage
		ram = {
			"id": f"{layer}_meta",
			"step": step,
			"match": "exact",
			"method": "index",
			"key_size": math.ceil(math.log2(num_ranges)),
			"data_size": len(classes)
		}
		ram_bits += num_ranges * len(classes)

		rmt.append(tcam)
		rmt.append(ram)

		step += 1

	return rmt, tcam_bits, ram_bits

x, tcam_bits, ram_bits = priority_aware(tree)
f = open("priority_aware.json", "w+")
f.write(json.dumps(x, indent=4))
f.close()

In [10]:
! command python3 ideal-rmt-simulator/sim.py priority_aware.json
print(f"TCAM bits: {tcam_bits}")
print(f"RAM bits:  {ram_bits}")

TCAM mapping: 
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SRAM mapping: 
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
id mapping: 
[['dst_range', 'dst_meta'], ['src_range', 'src_meta'], ['protocl_range', 'protocl_meta'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
TCAM bits: 2152
RAM bits:  504
